In [1]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
import scipy.stats as ss

### Heston Path Simulation using Monte-Carlo

It is governed by 2 set of S.D.E whose solution is as per below, note that dz1 and dz2 are correlated to each other by correlationo coefficient (rho), so, simulatenoeous simulation can be obtained using cholesky decomposition.


$$
\begin{equation*} 
    S_t = S_{t-1} e^{\left( r - \frac{\nu_t}{2} \right) dt + \sigma \sqrt{\nu_t} dZ_1}
\end{equation*}
$$

---

$$
\begin{equation*} 
    \nu_t = \nu_{t-1} + \kappa \left( \theta - \nu_{t-1} \right) dt + \sigma \sqrt{\nu_{t-1}}dZ_2
\end{equation*}
$$

In [2]:
# the below method gets v_t using a set of variables
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

In [3]:
# classic underlying stock price evolution
def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

In [4]:
def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

Assume these paramters are given for an example

In [5]:
v0 = 0.05
kappa_v = 3
sigma_v = 0.25
theta_v = 0.02
rho = -0.25

S0 = 100  # Current underlying asset price
r = 0.05  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 0.25  # Number of years
M = 50  # Total time steps
Ite = 10000  # Number of simulations
dt = T / M  # Length of time step

To account for dz1 and dz2 correlated with correlation rho

In [6]:
np.random.seed(0)
# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

In [7]:
np.random.seed(0)
# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

### Plotting the underlying SDE evolution path

In [ ]:
def plot_paths(n):
    fig = plt.figure(figsize=(18, 6))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    ax1.plot(range(len(S)), S[:, :n])
    ax1.grid()
    ax1.set_title("Heston Price paths")
    ax1.set_ylabel("Price")
    ax1.set_xlabel("Timestep")

    ax2.plot(range(len(V)), V[:, :n])
    ax2.grid()
    ax2.set_title("Heston Volatility paths")
    ax2.set_ylabel("Volatility")
    ax2.set_xlabel("Timestep")


plot_paths(100)

Heston model produces a distribution of returns that has heavier tails and kurtosis than normal distributions as what is observed in market for the stock returns.

In [8]:
def heston_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)
    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [10]:
print("European Call Price under Heston: ", heston_call_mc(S, 105, 0.05, 0.25, 0))

European Call Price under Heston:  2.146867253301489


In [ ]:
def heston_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :] )
    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
print("European put Price under Heston: ", heston_put_mc(S, 33, 0.05, 1, 0))